In [ ]:
import numpy as np
# import torch 
import terratorch

In [ ]:
test = {}
print(test.get('img_prefix'))

from osgeo import gdal
import rioxarray
testfile = r'C:\Users\timvd\Documents\Uni_2024-2025\UC\Project\ProjectCode\data\images\images\training\120x120\positive\198_2019-02-15T10_06_39.586Z_2.tif'
info = gdal.Info(testfile)
print(info)

# def open_tiff(fname):
#     data = rioxarray.open_rasterio(fname)
#     return data.to_numpy()

# open_tiff(testfile)

In [31]:
import os
from urllib.parse import urlparse

import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader,default_collate

from torchgeo.datasets import RasterDataset, stack_samples, unbind_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler,GeoSampler

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)

class Sentinel2(RasterDataset):
    filename_glob = '*.tif'
    # filename_regex = r'^.{6}_(?P<date>\d{8}T\d{6})_(?P<band>B0[\d])'
    # date_format = '%Y%m%dT%H%M%S'
    is_image = True
    separate_files = False
    all_bands = tuple([f'B0{i}' for i in range(1,14)])
    rgb_bands = ('B04', 'B03', 'B02')

    def plot(self, sample):
        # Find the correct band index order
        rgb_indices = []
        for band in self.rgb_bands:
            rgb_indices.append(self.all_bands.index(band))

        # Reorder and rescale the image
        image = sample['image'][rgb_indices].permute(1, 2, 0)
        image = torch.clamp(image / 10000, min=0, max=1).numpy()

        # Plot the image
        fig, ax = plt.subplots()
        ax.imshow(image)

        return fig

In [27]:
root = r'C:\Users\timvd\Documents\Uni_2024-2025\UC\Project\ProjectCode\data\images\images\training\120x120\positive'

dataset = Sentinel2(root)
print(dataset)

Sentinel2 Dataset
    type: GeoDataset
    bbox: BoundingBox(minx=-1585915.5339009934, maxx=1057978.5593479783, miny=4600624.042363878, maxy=6942656.59778398, mint=0.0, maxt=9.223372036854776e+18)
    size: 560


In [34]:
import torchgeo.datasets
help(torchgeo.datasets.stack_samples)

Help on function stack_samples in module torchgeo.datasets.utils:

stack_samples(samples: 'Iterable[Mapping[Any, Any]]') -> 'dict[Any, Any]'
    Stack a list of samples along a new axis.
    
    Useful for forming a mini-batch of samples to pass to
    :class:`torch.utils.data.DataLoader`.
    
    Args:
        samples: list of samples
    
    Returns:
        a single sample
    
    .. versionadded:: 0.2



In [ ]:
torch.manual_seed(1)
# root = r'C:\Users\timvd\Documents\Uni_2024-2025\UC\Project\ProjectCode\data\images\images\training\120x120\positive'
# dataset = Sentinel2(root)
# print(dataset)
sampler = RandomGeoSampler(dataset,size=40,length=1)#, size=40 , length=1,units='CRS')
# sampler = GeoSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler)#,collate_fn=stack_samples) #ADDING collate_fn=stack_samples CAUSES KERNEL TO DIE
print(dataloader)
for batch in dataloader:
    print(batch)
    sample = unbind_samples(batch)[0]
    dataset.plot(sample)
    plt.axis('off')
    plt.show()

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'rasterio.crs.CRS'>